In [1]:
import pandas as pd

books = pd.read_csv('books_with_categories.csv')

In [3]:
from transformers import pipeline

classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device="mps"
)
classifier("I love this!")

Device set to use mps


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764600355178118},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [4]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [6]:
classifier(books['description'][0])

[[{'label': 'fear', 'score': 0.6548413634300232},
  {'label': 'neutral', 'score': 0.1698521375656128},
  {'label': 'sadness', 'score': 0.11640870571136475},
  {'label': 'surprise', 'score': 0.020700665190815926},
  {'label': 'disgust', 'score': 0.01910073310136795},
  {'label': 'joy', 'score': 0.015161280520260334},
  {'label': 'anger', 'score': 0.0039351508021354675}]]

In [7]:
classifier(books['description'][0].split("."))

[[{'label': 'surprise', 'score': 0.7296023964881897},
  {'label': 'neutral', 'score': 0.14038589596748352},
  {'label': 'fear', 'score': 0.06816217303276062},
  {'label': 'joy', 'score': 0.04794251546263695},
  {'label': 'anger', 'score': 0.009156357496976852},
  {'label': 'disgust', 'score': 0.002628477755934},
  {'label': 'sadness', 'score': 0.0021221647039055824}],
 [{'label': 'neutral', 'score': 0.4493715167045593},
  {'label': 'disgust', 'score': 0.27359068393707275},
  {'label': 'joy', 'score': 0.10908294469118118},
  {'label': 'sadness', 'score': 0.09362741559743881},
  {'label': 'anger', 'score': 0.040478210896253586},
  {'label': 'surprise', 'score': 0.026970192790031433},
  {'label': 'fear', 'score': 0.006879056338220835}],
 [{'label': 'neutral', 'score': 0.6462160348892212},
  {'label': 'sadness', 'score': 0.24273338913917542},
  {'label': 'disgust', 'score': 0.04342262074351311},
  {'label': 'surprise', 'score': 0.028300566598773003},
  {'label': 'joy', 'score': 0.014211431

In [8]:
sentences = books['description'][0].split(".")
predictions = classifier(sentences)

In [9]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [10]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296023964881897},
 {'label': 'neutral', 'score': 0.14038589596748352},
 {'label': 'fear', 'score': 0.06816217303276062},
 {'label': 'joy', 'score': 0.04794251546263695},
 {'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.002628477755934},
 {'label': 'sadness', 'score': 0.0021221647039055824}]

In [14]:
sorted(predictions[0], key=lambda x:x['label'])

[{'label': 'anger', 'score': 0.009156357496976852},
 {'label': 'disgust', 'score': 0.002628477755934},
 {'label': 'fear', 'score': 0.06816217303276062},
 {'label': 'joy', 'score': 0.04794251546263695},
 {'label': 'neutral', 'score': 0.14038589596748352},
 {'label': 'sadness', 'score': 0.0021221647039055824},
 {'label': 'surprise', 'score': 0.7296023964881897}]

In [16]:
import numpy as np

emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
isbn = []

emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_prediction = sorted(prediction, key=lambda x:x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_prediction[index]['score'])
            
    return {label:np.max(scores) for label, scores in per_emotion_scores.items()}

In [17]:
for i in range(10):
    isbn.append(books['isbn13'][i])
    sentences = books['description'][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [19]:
emotion_scores

{'anger': [np.float64(0.06413353234529495),
  np.float64(0.6126179099082947),
  np.float64(0.06413353234529495),
  np.float64(0.35148343443870544),
  np.float64(0.08141229301691055),
  np.float64(0.2322252243757248),
  np.float64(0.5381841659545898),
  np.float64(0.06413353234529495),
  np.float64(0.3006708025932312),
  np.float64(0.06413353234529495)],
 'disgust': [np.float64(0.27359068393707275),
  np.float64(0.34828516840934753),
  np.float64(0.1040065586566925),
  np.float64(0.15072256326675415),
  np.float64(0.1844949573278427),
  np.float64(0.7271744608879089),
  np.float64(0.15585489571094513),
  np.float64(0.1040065586566925),
  np.float64(0.27948132157325745),
  np.float64(0.17792697250843048)],
 'fear': [np.float64(0.9281689524650574),
  np.float64(0.9425276517868042),
  np.float64(0.9723208546638489),
  np.float64(0.36070677638053894),
  np.float64(0.09504333138465881),
  np.float64(0.051362719386816025),
  np.float64(0.747428297996521),
  np.float64(0.40449604392051697),
  

In [20]:
from tqdm import tqdm

emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
isbn = []

emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books['isbn13'][i])
    sentences = books['description'][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [05:02<00:00, 17.15it/s]


In [21]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df['isbn13'] = isbn

In [22]:
emotions_df.head()

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612618,0.348285,0.942528,0.704422,0.887939,0.111690,0.252544,9780002261982
2,0.064134,0.104007,0.972321,0.767236,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765,9780006280934


In [23]:
books = pd.merge(books, emotions_df, on='isbn13')

In [24]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928169,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704422,0.887939,0.111690,0.252544
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.064134,0.104007,0.972321,0.767236,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765


In [25]:
books.to_csv('books_with_emotions.csv', index=False)